In [ ]:
def periodDistance(distance,L = 35000):
    if distance > L/2 :
        return  distance - L
    else:
        if distance < -L/2 :
            return distance + L
        else:
            return distance



In [ ]:
def pos_ori(haloID,snapNum,r_0,snapNum_fst=99):
    import numpy as np
    import illustris_python as il
    import random
    from functools import reduce

    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
    #SnapNum  = 99
    h = 0.677
    N_sample = 20000

    fields = ['SubhaloPos','SubhaloMass','SubhaloFlag']
    fields_tree = ['SubhaloPos','SnapNum']

    subhaloID  = il.groupcat.loadSingle(basePath, snapNum_fst, haloID=haloID)['GroupFirstSub']
    cen_tree   = il.sublink.loadTree(basePath,snapNum_fst,id=subhaloID,fields=fields_tree,
                                               onlyMPB=True)
    
    index = np.where(cen_tree['SnapNum']==snapNum)[0][0]
    #snapNum = cen_tree['SnapNum'][index]
    a = 1 / (1+SnapRedshift[snapNum])
    #f = a / h
    f = 1 / h  # use comoving coordinate: ckpc

    cen_pos = cen_tree['SubhaloPos'][index]
    data    = il.groupcat.loadSubhalos(basePath, snapNum, fields)
    
    par_pos = data['SubhaloPos']
    Mass    = data['SubhaloMass']
    #Flag    = data['SubhaloFlag']
    
    index_0 = selec_first(par_pos[:,0],cen_pos[0],r_0)
    index_1 = selec_first(par_pos[:,1],cen_pos[1],r_0)
    index_2 = selec_first(par_pos[:,2],cen_pos[2],r_0)
    
    index_first = reduce(np.intersect1d, [index_0, index_1, index_2])
    
#     if len(index_first) > N_sample:
#         array = random.sample(range(len(index_first)),N_sample)
#         index_first = index_first[array]

    par_pos = par_pos[index_first,:]

    Num_pars = len(par_pos[:,0])

    for i in np.arange(Num_pars):
        for j in np.arange(3):
            d = par_pos[i,j]-cen_pos[j]                    # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            par_pos[i,j] = cDistance * f                   # unit ckpc
            
    norm = np.linalg.norm(par_pos,axis=1)
    if len(norm)!=len(par_pos[:,0]):
        raise Exception('erro')
    
    index = np.where(norm <= r_0*f)[0]
    par_pos = par_pos[index,:]
    
    return par_pos,Mass[index_first[index]], index_first[index]

In [ ]:
def pos_ori_gal(haloID,snapNum,r_0,snapNum_fst=99):
    import numpy as np
    import illustris_python as il
    import random
    from functools import reduce

    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
    #SnapNum  = 99
    h = 0.677
    N_sample = 20000

    fields = ['SubhaloPos','SubhaloMassInRadType','SubhaloFlag']
    fields_tree = ['SubhaloPos','SnapNum']

    subhaloID  = il.groupcat.loadSingle(basePath, snapNum_fst, haloID=haloID)['GroupFirstSub']
    cen_tree   = il.sublink.loadTree(basePath,snapNum_fst,id=subhaloID,fields=fields_tree,
                                               onlyMPB=True)
    
    index = np.where(cen_tree['SnapNum']==snapNum)[0][0]
    #snapNum = cen_tree['SnapNum'][index]
    a = 1 / (1+SnapRedshift[snapNum])
    #f = a / h
    f = 1 / h  # use comoving coordinate: ckpc

    cen_pos = cen_tree['SubhaloPos'][index]
    data    = il.groupcat.loadSubhalos(basePath, snapNum, fields)
    
    par_pos = data['SubhaloPos']
    Mass    = data['SubhaloMassInRadType'][:,4]
    Flag    = data['SubhaloFlag']
    
    index_0 = selec_first(par_pos[:,0],cen_pos[0],r_0)
    index_1 = selec_first(par_pos[:,1],cen_pos[1],r_0)
    index_2 = selec_first(par_pos[:,2],cen_pos[2],r_0)
    
    index_first = reduce(np.intersect1d, [index_0, index_1, index_2])
    
#     if len(index_first) > N_sample:
#         array = random.sample(range(len(index_first)),N_sample)
#         index_first = index_first[array]

    par_pos = par_pos[index_first,:]

    Num_pars = len(par_pos[:,0])

    for i in np.arange(Num_pars):
        for j in np.arange(3):
            d = par_pos[i,j]-cen_pos[j]                    # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            par_pos[i,j] = cDistance * f                   # unit ckpc
            
    norm = np.linalg.norm(par_pos,axis=1)
    if len(norm)!=len(par_pos[:,0]):
        raise Exception('erro')
    
    index = np.where(norm <= r_0*f)[0]
    par_pos = par_pos[index,:]
    
    return par_pos,Mass[index_first[index]], Flag[index_first[index]], index_first[index]

In [ ]:
def selec_first(arr,x,r_0,L=35000):
    if x + r_0 > L:
        a = x+r_0-L
        b = x - r_0
        key = 0
    elif x - r_0 < 0:
        a = x+r_0
        b = x-r_0 +L
        key = 0
    else:
        a = x-r_0
        b = x+r_0
        key = 1
        
    if key:
        index = np.where((arr>a)&(arr<b))[0]
    else:
        index = np.where((arr<a)|(arr>b))[0]
        
    return index
    

In [ ]:
def fun_dm(haloID,SNums,r_0,snapNum_fst=99,m_th=4.5*(10**(-2))):
    import numpy as np
    POS_dm  = []
    GroupID = []
    Masses   = []
    i = 0
    h = 0.677
    m_th = m_th*h #unit: 10**10/h M_{\odot}
    
    for num in SNums:
        pos_dm,Mass,groupID = pos_ori(haloID,num,r_0,snapNum_fst=99)
        if len(pos_dm) != len(Mass):
            raise Exception('error')
            
        index = np.where(Mass>m_th)[0]
        pos_dm  = pos_dm[index]
        Mass    = Mass[index]
        groupID = groupID[index]
        
        POS_dm.append(pos_dm)
        GroupID.append(groupID)
        Masses.append(Mass)
        
        print(num)
        
    return POS_dm,GroupID,Masses

In [ ]:
def fun_gal(haloID,SNums,r_0,snapNum_fst=99,m_th=10**(-1.5)):
    import numpy as np
    from functools import reduce
    
    POS_dm  = []
    GroupID = []
    Masses   = []
    i = 0
    h = 0.677
    m_th = m_th*h #unit: 10**10/h M_{\odot}
    
    for num in SNums:
        pos_dm,Mass,Flag,groupID = pos_ori_gal(haloID,num,r_0,snapNum_fst=99)
        if len(pos_dm) != len(Mass):
            raise Exception('error')
            
        index_0 = np.where(Mass>m_th)[0]
        index_1 = np.where(Flag==1)[0]
        
        index = reduce(np.intersect1d, [index_0, index_1])
        
        pos_dm  = pos_dm[index]
        Mass    = Mass[index]
        groupID = groupID[index]
        
        POS_dm.append(pos_dm)
        GroupID.append(groupID)
        Masses.append(Mass)
        
        print(num)
        
    return POS_dm,GroupID,Masses

In [ ]:
def Transform_pos(sat_Trees,cen_Tree,SnapRedshift,snapNum):
    import numpy as np
    # used to plot RMS_N -N
    
    
    #SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    #fields = ['SnapNum','SubhaloPos','SubhaloVel','Group_R_Crit200']
    
    h = 0.677
    a = 1 / (1+SnapRedshift[snapNum])
    f2 = 1 / h
    Num_sats = len(sat_Trees)
    
    index_cen = np.where(cen_Tree['SnapNum']==snapNum)[0]
    
    pos_cen = cen_Tree['SubhaloPos'][index_cen,:].reshape(3,)
    #vel_cen = cen_Tree['SubhaloVel'][index_cen,:].reshape(3,)
    
    pos_sat = np.zeros([0,3])
    #vel_sat = np.zeros([0,3])
    
        
    for i in np.arange(Num_sats):
        index_sat = np.where(sat_Trees[i]['SnapNum']==snapNum)[0]
        try:
            pos_sat = np.append(pos_sat,sat_Trees[i]['SubhaloPos'][index_sat,:].reshape(1,3),axis=0)
            #vel_sat = np.append(vel_sat,sat_Trees[i]['SubhaloVel'][index_sat,:].reshape(1,3),axis=0)
        except ValueError:
            continue
            
    Num_sats = len(pos_sat[:,0])
            
    
    for i in np.arange(Num_sats):
        for j in np.arange(3):
            d = pos_sat[i,j]-pos_cen[j]                    # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            pos_sat[i,j] = cDistance * f2                  # unit ckpc
            #vel_sat[i,j] = vel_sat[i,j] - vel_cen[j]       # unit km/s
            
        
    return pos_sat
    

In [ ]:
def Draw_pos(cen_find_ID,sat_find_IDs,snapNum,basePath,num):    
    import numpy as np 
    import illustris_python as il
    import matplotlib.pyplot as plt
    from matplotlib import animation

    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']

    sat_Trees   = []
    fields_cen = ['SnapNum','SubhaloPos']
    fields_sat = ['SnapNum','SubhaloPos']

    cen_Tree = il.sublink.loadTree(basePath,snapNum,id=cen_find_ID,fields=fields_cen,
                                           onlyMPB=True)
    for Id in sat_find_IDs:
        sat_Trees.append(il.sublink.loadTree(basePath,snapNum,id=Id,fields=fields_sat,
                                           onlyMPB=True))
    #print(num)
    sat_class_pos = Transform_pos(sat_Trees,cen_Tree,SnapRedshift,snapNum=num)
    
        
    
    return sat_class_pos

In [ ]:
def fun_pos_14(haloID,SNums,basePath, snapNum,address):
    Data2 = dict(np.load(address))
    FoFGroup = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
    cen_find_ID = FoFGroup['GroupFirstSub']
    index = np.where(Data2['GroupID']==haloID)
    #print(index)
    sat_find_IDs = Data2['Sat_IDs'][index]
    sat_find_IDs = sat_find_IDs.reshape(sat_find_IDs.shape[1],)
    #print(sat_find_IDs)
    POS = []
    i = 0
    for num in SNums:
        pos = Draw_pos(cen_find_ID,sat_find_IDs,snapNum,basePath,num)
        POS.append(pos)
        
        print(num)
    return POS

In [ ]:
def rotationMat(x,y,z,theta):
    w = np.array([[0,-z,y],[z,0,-x],[-y,x,0]])
    I = np.identity(3)
    R = I + w*np.sin(theta) + np.dot(w,w)*(1-np.cos(theta))

    return R

In [ ]:
def R_1(ang_mome):
    ang_mome = ang_mome / np.linalg.norm(ang_mome)
    z  = np.array([0,0,1])
    v1 = np.cross(ang_mome,z)
    theta1 = np.arccos(np.dot(ang_mome,z))
    v1 = v1 / np.linalg.norm(v1)
    r1 = rotationMat(v1[0],v1[1],v1[2],theta1)
    
    return r1

## Large_Scale_Structure

### get data

In [ ]:
import numpy as np
import illustris_python as il

basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
h = 0.677
Address = ['/home/tnguser/data/infall_time_14.npz'] 


address= Address[0]
haloID = 395
SNums_sat = [14,24,30,99]
SNums_dm  = [14,24,30,99]
#SNums_gal = [99]

#SNums  = np.arange(10,31,1)
r_0    = 7000 * h    # unit: ckpc/h

POS_sat = fun_pos_14(haloID,SNums_sat,basePath, snapNum,address)
POS_dm,GroupID,Masses  = fun_dm(haloID,SNums_dm,r_0)
# POS_gal,GroupID_gal,Masses_gal  = fun_gal(haloID,SNums_gal,r_0)

# for i in np.arange(len(SNums_gal)):
#     POS_dm.append(POS_gal[i])
#     GroupID.append(GroupID_gal[i])
#     Masses.append(Masses_gal[i])

### rotate

In [ ]:
R_matrix = R_1(eigvec)
N = len(SNums_sat)
for i in np.arange(N):
    POS_sat[i] = np.dot(R_matrix,POS_sat[i].T).T
    POS_dm[i] = np.dot(R_matrix,POS_dm[i].T).T

### cut

In [ ]:
def cut(Arr,k,j,up,down,L_square=4000):
    import numpy as np
    from functools import reduce
    
    axis_n = np.delete(np.arange(3),j)
    #row = Arr.shape[0]
    if Arr.shape[1]!=3 or up<=down:
        raise Exception('erro')
    arr_0 = Arr[:,j]
    arr_1 = Arr[:,axis_n[0]]
    arr_2 = Arr[:,axis_n[1]]
    
    if True:
        index_0 = np.where((arr_0>down)&(arr_0<up))[0]
    else:
        index_0 = np.where((arr_0>(-L_square))&(arr_0<L_square))[0]
        
    index_1 = np.where((arr_1>(-L_square))&(arr_1<L_square))[0]
    index_2 = np.where((arr_2>(-L_square))&(arr_2<L_square))[0]
    
    index = reduce(np.intersect1d, [index_0, index_1, index_2])
    
    return Arr[index]

In [ ]:
POS_dm_cut = []
POS_sat_cut= []
length =1500
N = len(POS_dm)
for i in np.arange(N):
    Arr = POS_dm[i]
    Arr_anchor = POS_sat[i]
    for j in np.arange(3)[::-1]:
        anchor = np.median(Arr_anchor[:,j])
        POS_dm_cut.append(cut(Arr,(N-1-i),j,anchor+length,anchor-length))
        
for i in np.arange(3*N):
    POS_sat_cut.append(POS_sat[i//3])

### plot

In [ ]:
def circle(pos_x,pos_y,R):
    t = np.linspace(0,2*np.pi,300)

    x = pos_x + R * np.cos(t)
    y = pos_y + R * np.sin(t)
    
    return x,y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
from matplotlib.offsetbox import (AnchoredOffsetbox, AuxTransformBox, DrawingArea, TextArea, VPacker)

class AnchoredSizeBar(AnchoredOffsetbox):
    def __init__(self, transform, size, label, loc,
                 pad=0.1, borderpad=0.1, sep=2, prop=None, frameon=True):
        """
        Draw a horizontal bar with the size in data coordinate of the given
        axes. A label will be drawn underneath (center-aligned).

        pad, borderpad in fraction of the legend font size (or prop)
        sep in points.
        """
        self.size_bar = AuxTransformBox(transform)
        self.size_bar.add_artist(Line2D([0, size], [0, 0], color="blue"))
        self.txt_label = TextArea(label,textprops={'color':'blue'})
        self._box = VPacker(children=[self.size_bar, self.txt_label],
                            align="center",
                            pad=0, sep=sep)
        super().__init__(loc, pad=pad, borderpad=borderpad,
                         child=self._box, prop=prop, frameon=frameon)

def draw_sizebar(ax):
    """
    Draw a horizontal bar with length of 0.1 in data coordinates,
    with a fixed label underneath.
    """
    asb = AnchoredSizeBar(ax.transData,
                          1000,
                          r"$1\ \mathrm{cMpc}$",
                          loc='lower left',
                          pad=0, borderpad=0.5, sep=5,
                          frameon=False)
    ax.add_artist(asb)        


N_row = 4
N_column = 3

Group_pos = np.array([[-2094.14772224,1265.14731468,-979.95785514],
                       [ -171.10134095,-104.4342398,-764.0394909],
                       [0,0,0]]) 
Group_r   = np.array([510.31133519,169.9460479,166.46466614])

fig, Axs = plt.subplots(N_row, N_column, sharex=True,sharey=True,figsize=(9,12))
fig.subplots_adjust(hspace=0,wspace=0)
s = [r'$z=5.85$',r'$z=3.28$',r'$z=2.32$',r'$z=0$']
for i in np.arange(N_row):
    axs = Axs[i,:]
    if i!=N_row-1:
        axs[0].scatter(POS_dm_cut[i*3][:,0],POS_dm_cut[i*3][:,1],c='#000000',s=0.1)
        axs[1].scatter(POS_dm_cut[i*3+1][:,0],POS_dm_cut[i*3+1][:,2],c='#000000',s=0.1)
        axs[2].scatter(POS_dm_cut[i*3+2][:,1],POS_dm_cut[i*3+2][:,2],c='#000000',s=0.1)
    else:
        axs[0].scatter(POS_dm_cut[i*3][:,0],POS_dm_cut[i*3][:,1],c='#000000',s=0.1)
        axs[1].scatter(POS_dm_cut[i*3+1][:,0],POS_dm_cut[i*3+1][:,2],c='#000000',s=0.1)
        axs[2].scatter(POS_dm_cut[i*3+2][:,1],POS_dm_cut[i*3+2][:,2],c='#000000',s=0.1)
        
    axs[0].scatter(POS_sat_cut[i*3][:,0],POS_sat_cut[i*3][:,1],c='r',s=5)
    axs[1].scatter(POS_sat_cut[i*3+1][:,0],POS_sat_cut[i*3+1][:,2],c='r',s=5)
    axs[2].scatter(POS_sat_cut[i*3+2][:,1],POS_sat_cut[i*3+2][:,2],c='r',s=5)
    #axs[0].axis('equal')
    #axs[1].axis('equal')
    #axs[2].axis('equal')
    
    axs[0].tick_params(axis='y',right=False,labelright=False,left=False,labelleft=False)
    axs[1].tick_params(axis='y',right=False,labelright=False,left=False,labelleft=False)
    axs[2].tick_params(axis='y',right=False,labelright=False,left=False,labelleft=False)
    
#     axs[0].set_ylabel('y / kpc',fontsize=13,labelpad=0.01)
#     #axs[1].yaxis.set_label_position('right')
#     axs[1].set_ylabel('z / kpc',fontsize=13,labelpad=0.01)
#     axs[2].set_ylabel('z / kpc',fontsize=13,labelpad=0.01)
    
    axs[0].text(-4000,3700,s[i],weight='black',size='x-large')
    if i == N_row-1:
#         axs[0].set_xlabel('x / kpc',fontsize=13,labelpad=0.002)
#         axs[1].set_xlabel('x / kpc',fontsize=13,labelpad=0.002)
#         axs[2].set_xlabel('y / kpc',fontsize=13,labelpad=0.002)
        
        #axs[0].set_xticks(ticks=np.arange(-3000,3000,1500))
        axs[0].tick_params(axis='x',bottom=False,labelbottom=False,left=False,labelleft=False)
        axs[1].tick_params(axis='x',bottom=False,labelbottom=False,left=False,labelleft=False)
        axs[2].tick_params(axis='x',bottom=False,labelbottom=False,left=False,labelleft=False)
    if i ==1:
        #**kwargs={'color'='green'}
        axs[1].annotate('void',xy=(300,-1400),xycoords='data',xytext=(1300,-3500),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
        axs[2].annotate('void',xy=(-400,-900),xycoords='data',xytext=(-2900,-3000),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
        axs[1].annotate('void',xy=(800,1400),xycoords='data',xytext=(1300,3000),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
        axs[2].annotate('void',xy=(-1000,900),xycoords='data',xytext=(-3500,3000),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
        
    if i ==0:
        #**kwargs={'color'='green'}
        axs[1].annotate('void',xy=(300,-1400),xycoords='data',xytext=(1800,-3500),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
        axs[2].annotate('void',xy=(-1000,-1900),xycoords='data',xytext=(-2500,-4000),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
        
        axs[1].annotate('void',xy=(300,1400),xycoords='data',xytext=(1800,3500),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
        axs[2].annotate('void',xy=(-1000,900),xycoords='data',xytext=(-3000,3000),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
    if i ==2:
        #**kwargs={'color'='green'}
        axs[1].annotate('void',xy=(300,-900),xycoords='data',xytext=(1300,-2500),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
#         axs[2].annotate('void',xy=(-500,-1400),xycoords='data',xytext=(-2500,-3500),
#                         arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
#                        color='orange')
        
        axs[1].annotate('void',xy=(300,900),xycoords='data',xytext=(1300,3000),
                        arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
                       color='orange')
#         axs[2].annotate('void',xy=(-1000,900),xycoords='data',xytext=(-3000,3000),
#                         arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
#                        color='orange')
        
    if i ==3:
        #**kwargs={'color'='green'}
#         axs[1].annotate('void',xy=(-200,-2400),xycoords='data',xytext=(800,-4000),
#                         arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
#                        color='orange')
#         axs[2].annotate('void',xy=(-1000,-2400),xycoords='data',xytext=(-3000,-4000),
#                         arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
#                        color='orange')
        
#         axs[1].annotate('void',xy=(300,2800),xycoords='data',xytext=(1300,3500),
#                         arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
#                        color='orange')
#         axs[2].annotate('void',xy=(-500,2800),xycoords='data',xytext=(-2500,3500),
#                         arrowprops=dict(facecolor='orange',edgecolor='orange', shrink=0.05,width=2,headwidth=8),
#                        color='orange')
#         axs[0].scatter(galaxy_group_pos[0],galaxy_group_pos[1],c='#FF00FF',s=100,marker='*')
#         axs[1].scatter(POS_dm[0][0,0],POS_dm[0][0,2],c='b',s=50)
#         axs[2].scatter(galaxy_group_pos[1],galaxy_group_pos[2],c='#FF00FF',s=100,marker='*')
        
        for k in np.arange(3):
            x_0,y_0 = circle(Group_pos[k][0],Group_pos[k][1],Group_r[k])
            x_1,z_1 = circle(Group_pos[k][0],Group_pos[k][2],Group_r[k])
            y_2,z_2 = circle(Group_pos[k][1],Group_pos[k][2],Group_r[k])
            
            axs[0].plot(x_0,y_0)
            axs[1].plot(x_1,z_1)
            axs[2].plot(y_2,z_2)


        
    if i ==0:  
        axs[0].set_title('X-Y', fontsize=14)
        axs[1].set_title('X-Z', fontsize=14)
        axs[2].set_title('Y-Z', fontsize=14)
        
    if i == 3:
        draw_sizebar(axs[0])
        
    for j in np.arange(3):
        axs[j].spines['left'].set_linewidth(1.3)
        axs[j].spines['bottom'].set_linewidth(1.3)
        axs[j].spines['top'].set_linewidth(1.3)
        axs[j].spines['right'].set_linewidth(1.3)
        axs[j].scatter(0,0,c='springgreen',marker='x')
    
#plt.savefig('/home/tnguser/code/TheGreatPlane/Vedio_caratio/LSS_last_row_change.pdf',bbox_inches='tight',dpi=250)

## Zoom_in Plot

### 28 cMpc

In [ ]:
import numpy as np
import illustris_python as il

basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
h = 0.677
#Address = ['/home/tnguser/data/MWsatsL_Mstar_none_zero_low_pos_vel.npz'] 


#address= Address[0]
haloID = 395
#SNums_sat = [14,24,30,99]
SNums_dm  = [99]
#SNums_gal = [99]

#SNums  = np.arange(10,31,1)
r_0    = 24300 * h    # unit: ckpc/h
m_th   = 0 # unit: 10^10 M_{\odot}
#POS_sat = fun_pos(haloID,SNums_sat,basePath, snapNum,address)
POS_dm,GroupID,Masses  = fun_dm(haloID,SNums_dm,r_0,m_th=m_th)
#POS_gal,GroupID_gal,Masses_gal  = fun_gal(haloID,SNums_gal,r_0)

# for i in np.arange(len(POS_dm)):
#     POS_dm.append(POS_gal[i])
#     GroupID.append(GroupID_gal[i])
#     Masses.append(Masses_gal[i])

### rotate

In [ ]:
R_matrix = R_1(eigvec)
N = len(SNums_dm)
for i in np.arange(N):
    POS_dm[i] = np.dot(R_matrix,POS_dm[i].T).T

### cut

In [ ]:
def cut(Arr,k,j,up,down,L_square=4000):
    import numpy as np
    from functools import reduce
    
    axis_n = np.delete(np.arange(3),j)
    #row = Arr.shape[0]
    if Arr.shape[1]!=3 or up<=down:
        raise Exception('erro')
    arr_0 = Arr[:,j]
    arr_1 = Arr[:,axis_n[0]]
    arr_2 = Arr[:,axis_n[1]]
    
    if k:
        index_0 = np.where((arr_0>down)&(arr_0<up))[0]
    else:
        index_0 = np.where((arr_0>(-L_square))&(arr_0<L_square))[0]
        
    index_1 = np.where((arr_1>(-L_square))&(arr_1<L_square))[0]
    index_2 = np.where((arr_2>(-L_square))&(arr_2<L_square))[0]
    
    index = reduce(np.intersect1d, [index_0, index_1, index_2])
    
    return index

In [ ]:
POS_dm_cut  = []
Mass_dm_cut = []
length =1500         # unit: ckpc
L_square = 14000     # unit: ckpc
N = len(POS_dm)
for i in np.arange(N):
    Arr = POS_dm[i]
    for j in np.arange(3)[0:1]:
        anchor = 0
        index = cut(Arr,1,j,anchor+length,anchor-length,L_square)
        POS_dm_cut.append(Arr[index])
        Mass_dm_cut.append(Masses[0][index])
        

### CIC

In [ ]:
def Smooth2d(POS_particle,MASS_particle,L_x,L_y,Ng_x,Ng_y,M,L_square):
    import numpy as np
    
    if len(POS_particle.shape)!=2:
        raise Exception('error')
        
    r_x = L_x / Ng_x
    r_y = L_y / Ng_y
    
    pos_particle = np.zeros(POS_particle.shape)
    
    pos_particle[:,0] = (POS_particle[:,0]+L_square) / r_x
    pos_particle[:,1] = (POS_particle[:,1]+L_square) / r_y
    
    
    rho = np.zeros((Ng_x+1,Ng_y+1))
    
    Np = len(POS_particle[:,0])
    
    for index in np.arange(Np):
        xp = pos_particle[index,0]
        yp = pos_particle[index,1]
    
        mass = MASS_particle[index] / M
        
        i = int(xp)
        j = int(yp)
        
        dx = xp - i
        dy = yp - j
        
        tx = 1 - dx
        ty = 1 - dy
        
        rho[i,j]   =  rho[i,j] + mass*tx*ty
        
        rho[i,j+1] =  rho[i,j+1] + mass*tx*dy
        rho[i+1,j] =  rho[i+1,j] + mass*dx*ty
        
        rho[i+1,j+1] =  rho[i+1,j+1] + mass*dx*dy
        
    
    return rho

In [ ]:
POS_particle  = POS_dm_cut[0][:,1:3]
MASS_particle = np.ones(POS_dm_cut[0][:,0].shape)
#MASS_particle = Mass_dm_cut[0]
L_x = 2 * L_square
L_y = 2 * L_square
print(L_square)
Ng_x= 560  
Ng_y= 560
M = 1     # 10^10/h M_{\odot}
rho = Smooth2d(POS_particle,MASS_particle,L_x,L_y,Ng_x,Ng_y,M,L_square)

### 8cMpc

In [ ]:
import numpy as np
import illustris_python as il

basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
h = 0.677
Address = ['/home/tnguser/data/MWsatsL_Mstar_none_zero_low_pos_vel.npz'] 


address= Address[0]
haloID = 395

SNums_gal = [99]

#SNums  = np.arange(10,31,1)
r_0    = 14000 * h    # unit: ckpc/h

POS_gal,GroupID_gal,Masses_gal  = fun_gal(haloID,SNums_gal,r_0)


In [ ]:
R_matrix = R_1(eigvec)
N = len(SNums_gal)
for i in np.arange(N):
    POS_gal[i] = np.dot(R_matrix,POS_gal[i].T).T

In [ ]:
def cut(Arr,Arr1,L_square=4000):
    import numpy as np
    from functools import reduce
    
    if Arr.shape[1]!=3:
        raise Exception('erro')
    arr_0 = Arr[:,0]
    arr_1 = Arr[:,1]
    arr_2 = Arr[:,2]
    

    index_0 = np.where((arr_0>(-L_square))&(arr_0<L_square))[0]
    index_1 = np.where((arr_1>(-2*L_square))&(arr_1<2*L_square))[0]
    index_2 = np.where((arr_2>(-L_square))&(arr_2<L_square))[0]
    
    index = reduce(np.intersect1d, [index_0, index_1, index_2])
    
    return Arr[index],Arr1[index]

In [ ]:
POS_gal_cut  = []
Mass_gal_cut = []

N = len(POS_gal)
for i in np.arange(N):
    Arr_pos  = POS_gal[i]
    Arr_mass = Masses_gal[i]
    cut_pos,cut_mass = cut(Arr_pos,Arr_mass)
    index = np.argsort(cut_mass)
    POS_gal_cut.append(cut_pos[index])
    Mass_gal_cut.append(cut_mass[index])   

### 1cMpc

In [ ]:
import numpy as np
import illustris_python as il

basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
h = 0.677
Address = ['/home/tnguser/data/infall_time_14.npz'] 


address= Address[0]
haloID = 395

SNums_gal = [99]

#SNums  = np.arange(10,31,1)
r_0    = 3000 * h    # unit: ckpc/h
m_th   = 10**(-5)    # unit: 10**10 M_{\odot}

POS_gal,GroupID_gal,Masses_gal  = fun_gal(haloID,SNums_gal,r_0,m_th=m_th)



In [ ]:
R_matrix = R_1(eigvec)
N = len(SNums_gal)
for i in np.arange(N):
    POS_gal[i] = np.dot(R_matrix,POS_gal[i].T).T

In [ ]:
def cut_1(Arr,k,j,up,down,L_square=4000):
    import numpy as np
    from functools import reduce
    
    axis_n = np.delete(np.arange(3),j)
    #row = Arr.shape[0]
    if Arr.shape[1]!=3 or up<=down:
        raise Exception('erro')
    arr_0 = Arr[:,j]
    arr_1 = Arr[:,axis_n[0]]
    arr_2 = Arr[:,axis_n[1]]
    
    if k:
        index_0 = np.where((arr_0>down)&(arr_0<up))[0]
    else:
        index_0 = np.where((arr_0>(-L_square))&(arr_0<L_square))[0]
        
    index_1 = np.where((arr_1>(-L_square))&(arr_1<L_square))[0]
    index_2 = np.where((arr_2>(-L_square))&(arr_2<L_square))[0]
    
    index = reduce(np.intersect1d, [index_0, index_1, index_2])
    
    return index

In [ ]:
POS_gal_cut  = []
Mass_gal_cut = []

N = len(POS_gal)
for i in np.arange(N):
    Arr_pos  = POS_gal[i]
    Arr_mass = Masses_gal[i]
    k  = 1
    j  = 0
    up = 300
    down = -1000
    Index = cut_1(Arr_pos,k,j,up,down,L_square=1200)
    cut_pos  = Arr_pos[Index]
    cut_mass = Arr_mass[Index]
    index = np.argsort(cut_mass)
    POS_gal_cut.append(cut_pos[index])
    Mass_gal_cut.append(cut_mass[index])   

### 300cKpc

In [ ]:
import numpy as np
import illustris_python as il

basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
h = 0.677
Address = ['/home/tnguser/data/infall_time_14.npz'] 


address= Address[0]
haloID = 395
SNums_sat = [99]
SNums_gal = [99]

#SNums  = np.arange(10,31,1)
r_0    = 1000 * h    # unit: ckpc/h
m_th   = 10**(-7)    # unit: 10**10 M_{\odot}

POS_sat = fun_pos_14(haloID,SNums_sat,basePath, snapNum,address)
POS_gal,GroupID_gal,Masses_gal  = fun_gal(haloID,SNums_gal,r_0,m_th=m_th)


In [ ]:
def rotationMat(x,y,z,theta):
    w = np.array([[0,-z,y],[z,0,-x],[-y,x,0]])
    I = np.identity(3)
    R = I + w*np.sin(theta) + np.dot(w,w)*(1-np.cos(theta))

    return R

In [ ]:
theta = np.deg2rad(0)
R_matrix_1 = R_1(eigvec)
R_matrix_2 = rotationMat(0,0,1,theta)
R_matrix = np.matmul(R_matrix_2,R_matrix_1)
N = len(SNums_sat)
for i in np.arange(N):
    POS_sat[i] = np.dot(R_matrix,POS_sat[i].T).T
    POS_gal[i] = np.dot(R_matrix,POS_gal[i].T).T

In [ ]:
POS_gal_cut  = []
Mass_gal_cut = []

N = len(POS_gal)
for i in np.arange(N):
    Arr_pos  = POS_gal[i]
    Arr_mass = Masses_gal[i]
    cut_pos,cut_mass = cut(Arr_pos,Arr_mass,L_square=300)
    index = np.argsort(cut_mass)
    POS_gal_cut.append(cut_pos[index])
    Mass_gal_cut.append(cut_mass[index])   

### stellar disk

In [ ]:
def AngMome(basePath,snapNum,ID):
    
    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    a  = 1 / (1+SnapRedshift[snapNum])
    h  = 0.677
    f1 = 10**10 / h
    f2 = a / h
    
    fields = ['Coordinates','Velocities','GFM_StellarFormationTime','Masses']

    '''load position, and vercility of stars in the subhalo'''

    parData = il.snapshot.loadSubhalo(basePath,snapNum,ID,'star',fields=fields)

    '''find the the position and velocity of the subhalo'''

    subhaloInf = il.groupcat.loadSingle(basePath,snapNum,subhaloID=ID)
    subhaloPos = subhaloInf['SubhaloPos']
    subhaloVel = subhaloInf['SubhaloVel']
    starNum = subhaloInf['SubhaloLenType'][4]
    R_M = subhaloInf['SubhaloHalfmassRadType'][4] * f2     # kpc

    if starNum == 0:
        raise Exception('erro')
    
    '''reset the positions and verlosities '''

    for i in np.arange(starNum):
        for j in np.arange(3):
            d = parData['Coordinates'][i,j]-subhaloPos[j]  # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            parData['Coordinates'][i,j] = cDistance * f2    # unit kpc
            parData['Velocities'][i,j] = parData['Velocities'][i,j] * np.sqrt(a) - subhaloVel[j]  #km/s
            #parData['Velocities'][i,j] += H_z_0 * parData['Coordinates'][i,j] * 0.1  # change unit to km/s

    '''slect the real star particles and radius <3*R_M'''
    obj = []
    starRad = np.zeros(starNum)
    
    for i in np.arange(starNum): #get the radii of every star and wind particle
        starRad[i] = np.linalg.norm(parData['Coordinates'][i,:])

    parData['radius'] = starRad
    
    for i in np.arange(starNum):
        if parData['GFM_StellarFormationTime'][i] <= 0 or starRad[i] >= 6.3*R_M :   
            obj.append(i)

    fields = ['Coordinates','Velocities','GFM_StellarFormationTime','Masses','radius']
    for field in fields:
        parData[field] = np.delete(parData[field],obj,axis = 0)

    starNum = len(parData['GFM_StellarFormationTime']) # the number of real stars within 3*R_M
    if starNum == 0:
        raise Exception('erro')

    '''calculate the specific angular momentum'''

    angularMom = np.zeros([starNum,3])
    angularNorm = np.zeros(starNum)
    angularTotalMom = np.zeros(3)
    for i in np.arange(starNum):
        angularMom[i,:] = np.cross(parData['Coordinates'][i,:],parData['Velocities'][i,:])
        angularTotalMom = angularMom[i,:] + angularTotalMom

    angularTotalMom = angularTotalMom / np.linalg.norm(angularTotalMom)
    
    return angularTotalMom, parData['Coordinates'], parData['Masses']*f1

In [ ]:
def R_1(ang_mome):
    ang_mome = ang_mome / np.linalg.norm(ang_mome)
    z  = np.array([0,0,1])
    v1 = np.cross(ang_mome,z)
    theta1 = np.arccos(np.dot(ang_mome,z))
    v1 = v1 / np.linalg.norm(v1)
    r1 = rotationMat(v1[0],v1[1],v1[2],theta1)
    
    return r1

In [ ]:
basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
ID = 589873
ang_mome, star_pos, star_mass = AngMome(basePath,snapNum,ID)
R_Matrix = R_1(ang_mome)
c = np.dot(R_Matrix,star_pos.T).T
m = star_mass